# Discount Optimization

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.optimize import minimize

In [5]:
data = pd.read_csv('final_data.csv')
data = data.drop(['Product_Desc','depoid','Product_Description','unique_ID','Depo_Prod_Unique_ID','Product_Nickname','Nama_Depo'],axis=1)
print(data.shape)
data.head()

(278623, 22)


,Wholesaler_ID,Product_ID,PO_ID,Transaction_Date,Base_Price,Quantity_Sold,PPN,Discount_Type,Agg_Discount_Amount,modal_per_pcs_inc_PPN,...,decUomConversion1,decUomConversion2,Final_Price_New,Total_Revenue_New,discount_pct,Year_Transaction,Month_Transaction,is_conversion_equal,Convert_to_box,Price_Difference
0,0074-15-000007073,90103211101100,0074-23-000078054,2023-01-02,1083.333333,24,1981,Reguler,5405.405405,NaN,...,1,1,858.108108,20594.594587,0.207900,2023,1,True,1,225.225225
1,0074-19-000012078,90103211101100,0074-23-000078100,2023-01-02,1083.333333,24,1981,Reguler,5405.405405,NaN,...,1,1,858.108108,20594.594587,0.207900,2023,1,True,1,225.225225
2,0074-19-000012232,90103211101100,0074-23-000078099,2023-01-02,1083.333333,72,5945,Reguler,16216.216216,NaN,...,1,1,858.108108,61783.783760,0.207900,2023,1,True,3,225.225225
3,8133-16-000001011,90103211101100,0074-23-000078112,2023-01-02,1083.333333,24,1981,Reguler,5405.405405,NaN,...,1,1,858.108108,20594.594587,0.207900,2023,1,True,1,225.225225
4,0049-18-000006660,90103211101100,0049-23-000035048,2023-01-02,1125.000000,120,9662,Reguler,33784.684684,NaN,...,1,1,843.460961,101215.315316,0.250257,2023,1,True,5,281.539039


In [3]:
data = data.dropna()
data.shape

(210211, 29)

## feature engineering

In [4]:
drop_col = ['Product_Description', 'BOSnetszUomId', 'BOSnetdecUom', 'decUomConversion1', 'decUomConversion2','is_conversion_equal', 'Price_Difference']

data_purchased = data.drop(drop_col,axis=1)
data_purchased['Product_ID'] = data_purchased['Product_ID'].astype('str')
print(data_purchased.shape)
data_purchased.head()

(210211, 22)


,Wholesaler_ID,Product_ID,Product_Desc,PO_ID,Transaction_Date,Base_Price,Quantity_Sold,PPN,depoid,unique_ID,...,Depo_Prod_Unique_ID,Product_Nickname,Nama_Depo,modal_per_pcs_inc_PPN,Final_Price_New,Total_Revenue_New,discount_pct,Year_Transaction,Month_Transaction,Convert_to_box
5608,0074-19-000012078,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0074-23-000078100,2023-01-02,1083.333333,24,1981,74,90103211107100-0074-23-000078100,...,74-90103211107100,JDO5,DEPO MAKASSAR,727.187362,858.108108,20594.594587,0.207900,2023,1,1
5609,0049-18-000006660,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000035048,2023-01-02,1125.000000,120,9662,49,90103211107100-0049-23-000035048,...,49-90103211107100,JDO5,DEPO LOMBOK,742.104375,843.460961,101215.315316,0.250257,2023,1,5
5610,0049-18-000008369,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000035126,2023-01-03,1125.000000,48,3963,49,90103211107100-0049-23-000035126,...,49-90103211107100,JDO5,DEPO LOMBOK,742.104375,862.237237,41387.387388,0.233567,2023,1,2
5611,0049-13-000003123,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000035116,2023-01-03,1125.000000,120,9662,49,90103211107100-0049-23-000035116,...,49-90103211107100,JDO5,DEPO LOMBOK,742.104375,843.460961,101215.315316,0.250257,2023,1,5
5612,0074-17-000009593,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0074-23-000078212,2023-01-03,1083.333333,48,3964,74,90103211107100-0074-23-000078212,...,74-90103211107100,JDO5,DEPO MAKASSAR,727.187362,858.126877,41190.090075,0.207883,2023,1,2


### Create wholesaler-product column pair

In [5]:
data_purchased['Wholesaler_Product_ID'] = data_purchased['Wholesaler_ID'] + "-" + data_purchased['Product_ID']
data_purchased.head()

,Wholesaler_ID,Product_ID,Product_Desc,PO_ID,Transaction_Date,Base_Price,Quantity_Sold,PPN,depoid,unique_ID,...,Product_Nickname,Nama_Depo,modal_per_pcs_inc_PPN,Final_Price_New,Total_Revenue_New,discount_pct,Year_Transaction,Month_Transaction,Convert_to_box,Wholesaler_Product_ID
5608,0074-19-000012078,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0074-23-000078100,2023-01-02,1083.333333,24,1981,74,90103211107100-0074-23-000078100,...,JDO5,DEPO MAKASSAR,727.187362,858.108108,20594.594587,0.207900,2023,1,1,0074-19-000012078-90103211107100
5609,0049-18-000006660,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000035048,2023-01-02,1125.000000,120,9662,49,90103211107100-0049-23-000035048,...,JDO5,DEPO LOMBOK,742.104375,843.460961,101215.315316,0.250257,2023,1,5,0049-18-000006660-90103211107100
5610,0049-18-000008369,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000035126,2023-01-03,1125.000000,48,3963,49,90103211107100-0049-23-000035126,...,JDO5,DEPO LOMBOK,742.104375,862.237237,41387.387388,0.233567,2023,1,2,0049-18-000008369-90103211107100
5611,0049-13-000003123,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000035116,2023-01-03,1125.000000,120,9662,49,90103211107100-0049-23-000035116,...,JDO5,DEPO LOMBOK,742.104375,843.460961,101215.315316,0.250257,2023,1,5,0049-13-000003123-90103211107100
5612,0074-17-000009593,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0074-23-000078212,2023-01-03,1083.333333,48,3964,74,90103211107100-0074-23-000078212,...,JDO5,DEPO MAKASSAR,727.187362,858.126877,41190.090075,0.207883,2023,1,2,0074-17-000009593-90103211107100


### create profit, profit margin % col

In [6]:
# 1. Profit margin %
data_purchased['Profit_Margin_%'] = (
    (data_purchased['Final_Price_New'] - data_purchased['modal_per_pcs_inc_PPN'])
    / data_purchased['Final_Price_New'] )

# 2. Profit per unit
data_purchased['Profit_Per_Unit'] = data_purchased['Final_Price_New'] - data_purchased['modal_per_pcs_inc_PPN']

# 3. Total profit
data_purchased['Total_Profit'] = data_purchased['Profit_Per_Unit'] * data_purchased['Quantity_Sold']

data_purchased.head()

,Wholesaler_ID,Product_ID,Product_Desc,PO_ID,Transaction_Date,Base_Price,Quantity_Sold,PPN,depoid,unique_ID,...,Final_Price_New,Total_Revenue_New,discount_pct,Year_Transaction,Month_Transaction,Convert_to_box,Wholesaler_Product_ID,Profit_Margin_%,Profit_Per_Unit,Total_Profit
5608,0074-19-000012078,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0074-23-000078100,2023-01-02,1083.333333,24,1981,74,90103211107100-0074-23-000078100,...,858.108108,20594.594587,0.207900,2023,1,1,0074-19-000012078-90103211107100,0.152569,130.920745,3142.097887
5609,0049-18-000006660,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000035048,2023-01-02,1125.000000,120,9662,49,90103211107100-0049-23-000035048,...,843.460961,101215.315316,0.250257,2023,1,5,0049-18-000006660-90103211107100,0.120167,101.356586,12162.790316
5610,0049-18-000008369,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000035126,2023-01-03,1125.000000,48,3963,49,90103211107100-0049-23-000035126,...,862.237237,41387.387388,0.233567,2023,1,2,0049-18-000008369-90103211107100,0.139327,120.132862,5766.377388
5611,0049-13-000003123,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000035116,2023-01-03,1125.000000,120,9662,49,90103211107100-0049-23-000035116,...,843.460961,101215.315316,0.250257,2023,1,5,0049-13-000003123-90103211107100,0.120167,101.356586,12162.790316
5612,0074-17-000009593,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0074-23-000078212,2023-01-03,1083.333333,48,3964,74,90103211107100-0074-23-000078212,...,858.126877,41190.090075,0.207883,2023,1,2,0074-17-000009593-90103211107100,0.152588,130.939514,6285.096675


### Filter data so each wholesaler-product pair have more than 10 POs

In [7]:
data_purchased = data_purchased.groupby('Wholesaler_Product_ID').filter(lambda x : len(x)>=10)
print(data_purchased.shape)
print(len(list(data_purchased['Wholesaler_ID'].unique())))

(128660, 26)
1618


### Add price elasticity

In [164]:
# Calculate percentage changes in Quantity Sold and Final Price
data_purchased['Quantity_Change_%'] = data_purchased.groupby(['Wholesaler_ID', 'Product_ID'])['Quantity_Sold'].pct_change() * 100
data_purchased['Price_Change_%'] = data_purchased.groupby(['Wholesaler_ID', 'Product_ID'])['Final_Price_New'].pct_change() * 100

# Calculate Elasticity
data_purchased['Elasticity'] = data_purchased.apply(
    lambda row: row['Quantity_Change_%'] / row['Price_Change_%'] if row['Price_Change_%'] != 0 else np.nan,
    axis=1
)

# Fill NaN values (this will typically happen for the first row in each group)
data_purchased['Quantity_Change_%'] = data_purchased['Quantity_Change_%'].fillna(0)
data_purchased['Price_Change_%'] = data_purchased['Price_Change_%'].fillna(0)

# Cap elasticity to handle extreme outliers
data_purchased['Elasticity'] = data_purchased['Elasticity'].clip(-10, 10)

# minmax = MinMaxScaler()
# data_purchased['Elasticity'] = minmax.fit_transform(data_purchased[['Elasticity']])

data_purchased = data_purchased.dropna()
print(data_purchased.shape)
data_purchased.head()

(72245, 29)


,Wholesaler_ID,Product_ID,Product_Desc,PO_ID,Transaction_Date,Base_Price,Quantity_Sold,PPN,depoid,unique_ID,...,Year_Transaction,Month_Transaction,Convert_to_box,Wholesaler_Product_ID,Profit_Margin_%,Profit_Per_Unit,Total_Profit,Quantity_Change_%,Price_Change_%,Elasticity
5701,0049-18-000006660,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000036301,2023-01-16,1125.0,96,7729,49,90103211107100-0049-23-000036301,...,2023,1,4,0049-18-000006660-90103211107100,0.120175,101.364093,9730.952974,-20.000000,8.900842e-04,-10.0
5718,0049-08-000000754,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000036637,2023-01-19,1125.0,48,3864,49,90103211107100-0049-23-000036637,...,2023,1,2,0049-08-000000754-90103211107100,0.120175,101.364094,4865.476488,-60.000000,8.900866e-04,-10.0
5751,0049-20-000009207,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000037281,2023-01-24,1125.0,2400,255527,49,90103211107100-0049-23-000037281,...,2023,1,100,0049-20-000009207-90103211107100,0.312479,337.287517,809490.040541,-60.000000,-3.619910e+00,10.0
5756,0049-18-000007057,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000037424,2023-01-25,1125.0,4800,376576,49,90103211107100-0049-23-000037424,...,2023,1,200,0049-18-000007057-90103211107100,0.100152,82.595325,396457.558559,387.804878,-1.731393e-10,-10.0
5757,0049-18-000007057,90103211107100,JDO5/Okky Jelly Drink Mangga -150ml,0049-23-000037425,2023-01-25,1125.0,2400,188288,49,90103211107100-0049-23-000037425,...,2023,1,100,0049-18-000007057-90103211107100,0.100152,82.595325,198228.779280,-50.000000,2.524647e-11,-10.0


## Helper function

### ML modeling to predict quantity

In [11]:
def modeling(X, y):
    # split
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
    
    # encode
    encoder = TargetEncoder(cols=['Wholesaler_Product_ID'])  # or Wholesaler_Product_ID if you concatenate
    encoder.fit(X_train, y_train)

    X_train = encoder.transform(X_train)
    X_test = encoder.transform(X_test)

    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    r2_adjusted_train = 1 - (1-model.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
    r2_adjusted_test = 1 - (1-model.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

    print("R-squared (R²) Score:", r2)
    print("R² Adjusted Train Score:", r2_adjusted_train)
    print("R² Adjusted Test Score:", r2_adjusted_test)
    print(y_test.head())

    return {'encoder':encoder,
            'model':model,
            'r2':r2,
            'r2_adj_train': r2_adjusted_train,
            'r2_adj_test': r2_adjusted_test}

### simulate discount

In [12]:
def simulate_disc(pred_target, data, drop_col, target, disc, encoder, model):
  new_data = data.drop(drop_col+ target, axis=1)
  new_data['discount_pct'] = disc

  # calculate discount amount & final price manually because of new discount %
  new_data['Agg_Discount_Amount'] = new_data['Base_Price'] * new_data['discount_pct']
  new_data['Final_Price_New'] = new_data['Base_Price'] - new_data['Agg_Discount_Amount']
  new_data['Profit_Margin_%'] = (new_data['Final_Price_New'] - new_data['modal_per_pcs_inc_PPN']) / new_data['Final_Price_New']
  # Apply encodings
  new_data_encoded = encoder.transform(new_data)

  if pred_target == "revenue":
    # Predict
    predicted_revenue = model.predict(new_data_encoded)
    new_data['Predicted_Revenue'] = predicted_revenue
    new_data['Predicted_Quantity'] = new_data['Predicted_Revenue'] / new_data['Final_Price_New']

  if pred_target == "quantity":
      predicted_quantity = model.predict(new_data_encoded)
      new_data['Predicted_Quantity'] = predicted_quantity
      new_data['Predicted_Revenue'] = new_data['Predicted_Quantity'] * new_data['Final_Price_New']
      new_data['Predicted_Profit'] = new_data['Predicted_Quantity'] * (
          new_data['Final_Price_New'] - new_data['modal_per_pcs_inc_PPN']
      )

  return new_data

## Predict revenue

In [106]:
drop_col = ['Product_Desc', 'PO_ID', 'Wholesaler_ID','Product_ID','depoid', 'Elasticity','Depo_Prod_Unique_ID', 'Discount_Type','Nama_Depo', 'Transaction_Date','Convert_to_box', 'unique_ID', 'Product_Nickname', 'Year_Transaction']
    
target = ['Quantity_Sold']
X = data_purchased[:-50].drop(drop_col+target,axis=1)
val = data_purchased[-50:]
y = data_purchased[target][:-50]

In [107]:
result = modeling(X, y)
result

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


R-squared (R²) Score: 0.9982610020359585
R² Adjusted Train Score: 0.9998826707891303
R² Adjusted Test Score: 0.9982600549798455
        Quantity_Sold
133069             24
45250              72
34224             240
47726              72
17383              96


{'encoder': TargetEncoder(cols=['Wholesaler_Product_ID']),
 'model': RandomForestRegressor(random_state=42),
 'r2': 0.9982610020359585,
 'r2_adj_train': 0.9998826707891303,
 'r2_adj_test': 0.9982600549798455}

## Simulate hardcoded new discount

In [108]:
data_purchased[['Quantity_Sold','Total_Revenue_New','Final_Price_New','discount_pct','Profit_Margin_%','Total_Profit']][(data_purchased['Wholesaler_ID']=='0049-20-000009207') & (data_purchased['Product_ID']=='90103211107100')].head(1)

,Quantity_Sold,Total_Revenue_New,Final_Price_New,discount_pct,Profit_Margin_%,Total_Profit
5664,2400,2.687838e+06,1119.932432,0.004505,0.337367,906787.337838


customer cukup reaktif terhadap perubahan harga, diskon awal hanya 0.4%, lalu diberikan diskon 2%

In [109]:
data = data_purchased[(data_purchased['Wholesaler_ID']=='0049-20-000009207') & (data_purchased['Product_ID']=='90103211107100')].head(1)
simulate_disc("quantity",data,drop_col,target,0.02, result.get('encoder'), result.get('model'))

,Base_Price,PPN,Agg_Discount_Amount,modal_per_pcs_inc_PPN,Final_Price_New,Total_Revenue_New,discount_pct,Month_Transaction,Wholesaler_Product_ID,Profit_Margin_%,Profit_Per_Unit,Total_Profit,Quantity_Change_%,Price_Change_%,Predicted_Quantity,Predicted_Revenue,Predicted_Profit
5664,1125.0,266229,22.5,742.104375,1102.5,2.687838e+06,0.02,1,0049-20-000009207-90103211107100,0.326889,377.828057,906787.337838,0.0,0.0,2511.6,2769039.0,905169.65175


## Find optimized discount

using minimize() function from scipy to find the discount that maximizes profit

Since we want to maximize profit, we convert Maximize profit to Minimize (–profit)

It tries to find the x (the discount) that minimizes f(x) using iterative methods like SLSQP (Sequential Least Squares Programming)

1. Input to minimize():
A discount value x[0]

2. Objective function:
f(x) to minimize -profit by calculating predicted quantity using regression coefficients

3. Computes final price from discount: Calculates profit = (final price – cost) × quantity then returns –profit (because we want to maximize it)

4. Constraints:

- Discount must be between 0.5% and 45%

- Final price must be ≥ cost

Why minimize and not linear programming?

D = discount

- Final price = Base_Price * (1 - D) → linear in D

- But profit = (Final price - cost) * Predicted quantity → not constant because profit is dependant to quantity

This is a product of two linear terms in D, which results in a nonlinear (specifically, quadratic) function.
That means the objective function is not linear, so linear programming can't be used.



In [156]:
results = []

# Group the data to optimize each Wholesaler_Product_ID separately
grouped = data_purchased.groupby('Wholesaler_Product_ID')

for key, group in grouped:
    if group['discount_pct'].nunique() < 2:
        continue  # Skip if not enough variation in discount

    # Separate features and target
    X = group.drop(columns=drop_col+target)
    y = group['Quantity_Sold']

    # Encode categorical (if needed)
    if 'Wholesaler_Product_ID' in X.columns:
        encoder = TargetEncoder(cols=['Wholesaler_Product_ID'])
        X = encoder.fit_transform(X, y)

    # Fit regression
    # Using linear regression to stimate how Quantity_Sold responds to discount_pct + other variables
    reg = LinearRegression()
    reg.fit(X, y)

    # To evaluate what happens if we change the discount, we need to recalculate the predicted quantity
    # use intercept & coef as a way to predict a new value given changes in features (like a different discount)
    intercept = reg.intercept_
    coef = dict(zip(X.columns, reg.coef_))

    # Use latest price and cost from group
    base_price = group['Base_Price'].iloc[-1]
    cost = group['modal_per_pcs_inc_PPN'].iloc[-1]

    # Fixed values for features (other than discount_pct)
    latest_features = X.iloc[-1].to_dict()

    # define objective function
    # Returns negative profit so that minimize() can maximize it
    def objective(x):
        D = x[0]
        features = latest_features.copy()
        features['discount_pct'] = D
        predicted_quantity = intercept + sum(coef[k] * features[k] for k in coef)
        final_price = base_price * (1 - D)
        profit = (final_price - cost) * predicted_quantity
        return -profit

    # Constraints: 0.5% ≤ D ≤ 45% and final price ≥ cost
    constraints = [
        {'type': 'ineq', 'fun': lambda x: x[0] - 0.005},
        {'type': 'ineq', 'fun': lambda x: 0.45 - x[0]},
        {'type': 'ineq', 'fun': lambda x: base_price * (1 - x[0]) - cost}
    ]

    opt_result = minimize(objective, x0=[0.45], constraints=constraints, method='SLSQP')

    if opt_result.success:
        # Extract wholesaler and product IDs from the group
        wholesaler_id = group['Wholesaler_ID'].iloc[0]
        product_id = group['Product_ID'].iloc[0]
        opt_disc = round(opt_result.x[0], 3)
        base_price = group['Base_Price'].iloc[0]
        cost = group['modal_per_pcs_inc_PPN'].iloc[0]
        past_price = group['Final_Price_New'].iloc[0]
        past_discount = group['discount_pct'].iloc[0]
        past_qty = group['Quantity_Sold'].iloc[0]
        past_profit = group['Total_Profit'].iloc[0]

        features = latest_features.copy()
        features['discount_pct'] = opt_disc
        predicted_quantity = intercept + sum(coef[k] * features[k] for k in coef)

        optimized_discount_amt = base_price * opt_disc
        new_final_price = base_price - optimized_discount_amt
        profit_margin_pct = (new_final_price - cost) / new_final_price
        profit_per_unit = new_final_price - cost
        total_profit = profit_per_unit * predicted_quantity

        results.append({
            'Wholesaler_ID': wholesaler_id,
            'Product_ID': product_id,
            'past_price': past_price,
            'past_discount': past_discount,
            'past_qty': past_qty,
            'past_profit':past_profit,
            'optimized_discount': opt_disc,
            'optimized_discount_amt': optimized_discount_amt,
            'opt_predicted_quantity': predicted_quantity,
            'opt_predicted_profit':total_profit,
            'new_final_price': new_final_price
        })

# Convert results to DataFrame
optimized_df = pd.DataFrame(results)

In [157]:
# discount structure
optimized_df

,Wholesaler_ID,Product_ID,past_price,past_discount,past_qty,past_profit,optimized_discount,optimized_discount_amt,opt_predicted_quantity,opt_predicted_profit,new_final_price
0,0049-08-000000070,90103211116100,1600.000000,0.000000,24,7075.000800,0.014,22.400000,24.0,6537.40080,1577.600000
1,0049-08-000000114,90103211107100,824.703453,0.266930,240,19823.778830,0.011,12.375000,120.0,44462.47500,1112.625000
2,0049-08-000000114,90103211143100,799.924925,0.085800,240,13876.931982,0.037,32.375000,72.0,7237.48500,842.625000
3,0049-08-000000114,90103211144100,799.924925,0.085800,240,13876.931982,0.037,32.375000,72.0,7237.48500,842.625000
4,0049-08-000000625,90103211142100,1364.864865,0.090090,120,7158.787785,0.014,21.000000,24.0,4171.00080,1479.000000
...,...,...,...,...,...,...,...,...,...,...,...
848,8133-17-000001367,90103211116100,1472.972973,0.018018,96,16105.408606,0.014,21.000000,24.0,4171.00080,1479.000000
849,8133-17-000001367,90103211122100,1472.972973,0.018018,96,16105.408606,0.014,21.000000,24.0,4171.00080,1479.000000
850,8133-17-000001367,90103211154100,1418.918919,0.054054,48,5458.109709,0.014,21.000000,24.0,4171.00080,1479.000000
851,8133-17-000001369,90103311139100,784.534534,0.099099,360,20644.981813,0.005,4.354167,240.0,33430.03292,866.479166


## Compare the result of optimized discount to random forest result

applying the optimized discount to random forest model to predict quantity & profit

In [158]:
comparison = optimized_df.copy()

there's a significant difference between opt_predicted_quantity and rf_predicted_quantity,

because opt_predicted_quantity uses linear regression, while rf_predicted_quantity use random forest, which learns the pattern better

In [159]:
for i in range(comparison.shape[0]):
    row = comparison.iloc[i]
    wholesaler_id = row['Wholesaler_ID']
    product_id = row['Product_ID']
    optim_disc = row['optimized_discount']

    subset = data_purchased[
        (data_purchased['Wholesaler_ID'] == wholesaler_id) &
        (data_purchased['Product_ID'] == product_id)
    ]

    res = simulate_disc("quantity", subset, drop_col, target, optim_disc, result.get('encoder'), result.get('model'))

    # Use first predicted value (assumes one row or that the first is representative)
    # comparison.at[i, 'discount_act'] = subset['discount_pct'].iloc[0]
    # comparison.at[i, 'quantity_sold_act'] = subset['Quantity_Sold'].iloc[0]
    # comparison.at[i, 'profit_act'] = subset['Total_Profit'].iloc[0]
    comparison.at[i, 'rf_predicted_quantity'] = res['Predicted_Quantity'].iloc[0]
    comparison.at[i, 'rf_predicted_profit'] = res['Predicted_Profit'].iloc[0]

comparison

,Wholesaler_ID,Product_ID,past_price,past_discount,past_qty,past_profit,optimized_discount,optimized_discount_amt,opt_predicted_quantity,opt_predicted_profit,new_final_price,rf_predicted_quantity,rf_predicted_profit
0,0049-08-000000070,90103211116100,1600.000000,0.000000,24,7075.000800,0.014,22.400000,24.0,6537.40080,1577.600000,24.00,6537.400800
1,0049-08-000000114,90103211107100,824.703453,0.266930,240,19823.778830,0.011,12.375000,120.0,44462.47500,1112.625000,193.44,71673.509700
2,0049-08-000000114,90103211143100,799.924925,0.085800,240,13876.931982,0.037,32.375000,72.0,7237.48500,842.625000,240.00,24124.950000
3,0049-08-000000114,90103211144100,799.924925,0.085800,240,13876.931982,0.037,32.375000,72.0,7237.48500,842.625000,240.00,24124.950000
4,0049-08-000000625,90103211142100,1364.864865,0.090090,120,7158.787785,0.014,21.000000,24.0,4171.00080,1479.000000,119.28,20729.873976
...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,8133-17-000001367,90103211116100,1472.972973,0.018018,96,16105.408606,0.014,21.000000,24.0,4171.00080,1479.000000,96.00,16684.003200
849,8133-17-000001367,90103211122100,1472.972973,0.018018,96,16105.408606,0.014,21.000000,24.0,4171.00080,1479.000000,96.00,16684.003200
850,8133-17-000001367,90103211154100,1418.918919,0.054054,48,5458.109709,0.014,21.000000,24.0,4171.00080,1479.000000,48.00,8342.001600
851,8133-17-000001369,90103311139100,784.534534,0.099099,360,20644.981813,0.005,4.354167,240.0,33430.03292,866.479166,352.08,49041.858294


In [160]:
comparison['qty_difference'] = comparison['past_qty'] - comparison['rf_predicted_quantity']
comparison['profit_difference'] = comparison['past_profit'] - comparison['rf_predicted_profit']

In [161]:
# predicted higher than actual = optimized discount might work
comparison[comparison['profit_difference']<0]

,Wholesaler_ID,Product_ID,past_price,past_discount,past_qty,past_profit,optimized_discount,optimized_discount_amt,opt_predicted_quantity,opt_predicted_profit,new_final_price,rf_predicted_quantity,rf_predicted_profit,qty_difference,profit_difference
1,0049-08-000000114,90103211107100,824.703453,0.266930,240,19823.778830,0.011,12.375000,120.0,44462.47500,1112.625000,193.44,71673.509700,46.56,-51849.730870
2,0049-08-000000114,90103211143100,799.924925,0.085800,240,13876.931982,0.037,32.375000,72.0,7237.48500,842.625000,240.00,24124.950000,0.00,-10248.018018
3,0049-08-000000114,90103211144100,799.924925,0.085800,240,13876.931982,0.037,32.375000,72.0,7237.48500,842.625000,240.00,24124.950000,0.00,-10248.018018
4,0049-08-000000625,90103211142100,1364.864865,0.090090,120,7158.787785,0.014,21.000000,24.0,4171.00080,1479.000000,119.28,20729.873976,0.72,-13571.086191
5,0049-08-000000711,90103211116100,1351.351351,0.099099,120,5537.166164,0.039,58.500000,72.0,9813.00240,1441.500000,119.28,16256.873976,0.72,-10719.707812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,8133-17-000001367,90103211116100,1472.972973,0.018018,96,16105.408606,0.014,21.000000,24.0,4171.00080,1479.000000,96.00,16684.003200,0.00,-578.594594
849,8133-17-000001367,90103211122100,1472.972973,0.018018,96,16105.408606,0.014,21.000000,24.0,4171.00080,1479.000000,96.00,16684.003200,0.00,-578.594594
850,8133-17-000001367,90103211154100,1418.918919,0.054054,48,5458.109709,0.014,21.000000,24.0,4171.00080,1479.000000,48.00,8342.001600,0.00,-2883.891891
851,8133-17-000001369,90103311139100,784.534534,0.099099,360,20644.981813,0.005,4.354167,240.0,33430.03292,866.479166,352.08,49041.858294,7.92,-28396.876481


In [162]:
# actual higher than predicted, optimized discount might need to be further discussed
comparison[comparison['profit_difference']>0]

,Wholesaler_ID,Product_ID,past_price,past_discount,past_qty,past_profit,optimized_discount,optimized_discount_amt,opt_predicted_quantity,opt_predicted_profit,new_final_price,rf_predicted_quantity,rf_predicted_profit,qty_difference,profit_difference
0,0049-08-000000070,90103211116100,1600.000000,0.000000,24,7075.000800,0.014,22.400000,24.0,6537.400800,1577.600000,24.0,6537.400800,0.0,537.600000
13,0049-08-000001291,90103211142100,1472.972973,0.018018,24,4026.352152,0.038,57.000000,48.0,6614.001601,1443.000000,24.0,3307.000800,0.0,719.351352
17,0049-09-000001697,90103211116100,1500.000000,0.000000,48,9350.001600,0.014,21.000000,24.0,4171.000800,1479.000000,48.0,8342.001600,0.0,1008.000000
18,0049-09-000001697,90103211121100,1500.000000,0.000000,24,4675.000800,0.014,21.000000,24.0,4171.000800,1479.000000,24.0,4171.000800,0.0,504.000000
31,0049-13-000003060,90103211116100,1500.000000,0.000000,24,4675.000800,0.014,21.000000,24.0,4171.000800,1479.000000,24.0,4171.000800,0.0,504.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,8133-16-000000915,90103311140100,870.833333,0.000000,24,3447.503292,0.006,5.225000,48.0,6644.206584,865.608333,24.0,3322.103292,0.0,125.400000
844,8133-16-000000938,90103311133100,870.833333,0.000000,48,6895.006584,0.006,5.225000,48.0,6644.206584,865.608333,48.0,6644.206584,0.0,250.800000
845,8133-16-000001001,90103311133100,808.070570,0.072072,240,19411.969857,0.106,92.308333,96.0,4928.413171,778.525000,240.0,12321.032928,0.0,7090.936929
846,8133-16-000001011,90103211116100,1500.000000,0.000000,24,4675.000800,0.014,21.000000,24.0,4171.000800,1479.000000,24.0,4171.000800,0.0,504.000000
